## Chapter7. 합성곱 신경방(CNN)

## 구현
* filter_num : 필터수
* filter_size : 필터사이즈
*
*
*
*
* weight_init_std : 초기화 때의 가중치 초기 표준편차
*

In [34]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

from dataset.mnist import load_mnist
from common.trainer import Trainer


In [40]:

class SimpleConvNet:
    """단순한 합성곱 신경망
    
    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """손실 함수를 구한다.
        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """기울기를 구한다（수치미분）.
        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).
        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

In [11]:
def filter_show(filters, nx=8, margin=3, scale=10):
    """
    c.f. https://gist.github.com/aidiary/07d530d5e08011832b12#file-draw_weight-py
    """
    FN, C, FH, FW = filters.shape
    ny = int(np.ceil(FN / nx))

    fig = plt.figure()
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    for i in range(FN):
        ax = fig.add_subplot(ny, nx, i+1, xticks=[], yticks=[])
        ax.imshow(filters[i, 0], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.show()


### CNN 실행코드

In [41]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)
max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28),
                       conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                       hidden_size=100,output_size=10,weight_init_std=0.01)

trainer = Trainer(network, x_train, t_train, x_test, t_test,
                 epochs=max_epochs, mini_batch_size=100,
                 optimizer='Adam', optimizer_param={'lr':0.001},
                 evaluate_sample_num_per_epoch=10000)

trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.3002128278
=== epoch:1, train acc:0.2679, test acc:0.2572 ===
train loss:2.29815949978
train loss:2.29498954562
train loss:2.28846875312
train loss:2.28552308324
train loss:2.27189952304
train loss:2.25896402054
train loss:2.25629842933
train loss:2.22734698672
train loss:2.1850987856
train loss:2.16383642327
train loss:2.11617502304
train loss:2.07242906391
train loss:2.06338961451
train loss:1.95600609156
train loss:1.92739461371
train loss:1.91972703941
train loss:1.79688182472
train loss:1.75149695139
train loss:1.66016201322
train loss:1.50342660152
train loss:1.53514796227
train loss:1.43402933739
train loss:1.33331561934
train loss:1.37982873438
train loss:1.23552669681
train loss:1.14954827926
train loss:0.966318790133
train loss:1.0672653144
train loss:0.998089233778
train loss:0.952236139593
train loss:0.856143961333
train loss:0.795819507765
train loss:0.778462655624
train loss:0.837478597424
train loss:0.794489238373
train loss:0.696991811395
train loss:0.64064

train loss:0.130381014611
train loss:0.182447941139
train loss:0.176095563993
train loss:0.272944522459
train loss:0.16406618227
train loss:0.188147604627
train loss:0.173988662419
train loss:0.161926357653
train loss:0.0827945645132
train loss:0.171275104769
train loss:0.137053336325
train loss:0.326543807929
train loss:0.167200191605
train loss:0.100787600951
train loss:0.167878543475
train loss:0.193651124203
train loss:0.118592265274
train loss:0.108072107817
train loss:0.114822556084
train loss:0.161812754743
train loss:0.198601586826
train loss:0.221674767693
train loss:0.194738910536
train loss:0.117516786875
train loss:0.18136440556
train loss:0.171605228019
train loss:0.16805896987
train loss:0.101713367725
train loss:0.302243452083
train loss:0.113556210818
train loss:0.146305706488
train loss:0.183229157143
train loss:0.191984028646
train loss:0.244097603649
train loss:0.259831833544
train loss:0.211124345014
train loss:0.137829094841
train loss:0.200477490766
train loss:0.1

train loss:0.0626098289317
train loss:0.0371275892199
train loss:0.0924150151615
train loss:0.0517047794083
train loss:0.0620467856577
train loss:0.0894260859319
train loss:0.121837163652
train loss:0.0459292082591
train loss:0.0827573361242
train loss:0.113183635642
train loss:0.0911672419733
train loss:0.0845649579807
train loss:0.0824371394498
train loss:0.0361734089742
train loss:0.0579533933539
train loss:0.0953386239217
train loss:0.101643138881
train loss:0.0622832594689
train loss:0.12290055441
train loss:0.0447681109428
train loss:0.154742296521
train loss:0.164039385596
train loss:0.101685165265
train loss:0.135356735895
train loss:0.0630568864589
train loss:0.0910515885337
train loss:0.0922590902761
train loss:0.141124344745
train loss:0.0806584856527
train loss:0.0894434671813
train loss:0.112424847092
train loss:0.0970304511056
train loss:0.0584499382376
train loss:0.0602413526138
train loss:0.0719245394966
train loss:0.0919545115716
train loss:0.125967708262
train loss:0.

train loss:0.0597686631784
train loss:0.0649341576539
train loss:0.100428468682
train loss:0.142773769029
train loss:0.131321720165
train loss:0.0768687029047
train loss:0.114493119218
train loss:0.0950633801436
train loss:0.0875584419048
train loss:0.0252048972341
train loss:0.0640612531768
train loss:0.128714164613
train loss:0.0632258141598
train loss:0.0485846617829
train loss:0.0566776251632
train loss:0.0858233544128
train loss:0.0239264296562
train loss:0.0577260494463
train loss:0.0777789402926
train loss:0.143310166088
train loss:0.128093068516
train loss:0.0373486468777
train loss:0.125765126419
train loss:0.0621839338081
train loss:0.1243804289
train loss:0.12786460834
train loss:0.0811545093781
train loss:0.120433369374
train loss:0.111231677249
train loss:0.0672763476639
train loss:0.040433112553
train loss:0.0329988762872
train loss:0.0270473223679
train loss:0.106884196812
train loss:0.150708236976
train loss:0.0463509608825
train loss:0.111519430257
train loss:0.0239324

train loss:0.0518053195019
train loss:0.0504815479835
train loss:0.0381459757959
train loss:0.030739536409
train loss:0.0639753752112
train loss:0.0190385824003
train loss:0.0915467942565
train loss:0.0719722558435
train loss:0.058730664442


KeyboardInterrupt: 